
# Coin

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")


import pandas as pd
import datetime as dt

In [2]:
bt = pd.read_csv("data/bitc.csv")
et = pd.read_csv("data/ethereum.csv")
usd = pd.read_csv("data/usdc.csv")

In [3]:
bt.rename(columns={'timestamp':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume': 'Volume','marketCap': 'Marketcap'}, inplace=True)
et.rename(columns={'timestamp':'Date','open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume': 'Volume','marketCap': 'Marketcap'}, inplace=True)
usd.rename(columns={'timestamp':'Date','open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume': 'Volume','marketCap': 'Marketcap'}, inplace=True)

In [4]:
bt['Date'] = pd.to_datetime(bt['Date'])
et['Date'] = pd.to_datetime(et['Date'])
usd['Date'] = pd.to_datetime(usd['Date'])

In [5]:
bt = bt[bt['Date'].between('2021-02-01','2024-04-30')]
et = et[et['Date'].between('2021-02-01','2024-04-30')]
usd = usd[usd['Date'].between('2021-02-01','2024-04-30')]

In [6]:
bt['Vol/cap'] = bt['Volume'] / bt['Marketcap']
et['Vol/cap'] = et['Volume'] / et['Marketcap']
usd['Vol/cap'] = usd['Volume'] / usd['Marketcap']

In [7]:
bt['HL_spread'] = bt['High'] - bt['Low']
bt['OC_spread'] = bt['Open'] - bt['Close']

et['HL_spread'] = et['High'] - et['Low']
et['OC_spread'] = et['Open'] - et['Close']

usd['HL_spread'] = usd['High'] - usd['Low']
usd['OC_spread'] = usd['Open'] - usd['Close']

In [8]:
bt.to_csv('data/bitcoin_price.csv', index=False)
et.to_csv('data/ether_price.csv', index=False)
usd.to_csv('data/usd_price.csv', index=False)

In [9]:
bt = pd.read_csv("data/bitcoin_price.csv")
et = pd.read_csv("data/ether_price.csv")
usd = pd.read_csv("data/usd_price.csv")

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()

In [12]:
bts = bt.set_index('Date',drop=True)
ets = et.set_index('Date',drop=True)
usds = usd.set_index('Date',drop=True)

In [13]:
bitcoin = pd.DataFrame(scaler.fit_transform(bts), columns = bts.columns)
ether = pd.DataFrame(scaler.fit_transform(ets), columns = ets.columns)
usdc = pd.DataFrame(scaler.fit_transform(usds), columns = usds.columns)

In [14]:
bitcoin.index = bt['Date']
ether.index = bt['Date']
usdc.index = bt['Date']

In [15]:
bitcoin.to_csv('data/bitcoin_norm.csv')
ether.to_csv('data/ether_norm.csv')
usdc.to_csv('data/usdc_norm.csv')

# News

In [16]:
news = pd.read_csv("data/news_data_all.csv")

In [17]:
news.drop(news[news['Date']=='Date'].index, axis=0, inplace=True)
news['Date'] = pd.to_datetime(news['Date'])
news = news[news['Date'].between('2021-02-01','2024-04-30')]
news = news.sort_values(by='Date', ascending=False)
news = news.reset_index(drop=True)

In [18]:
news.to_csv('data/news.csv', index=False)

In [19]:
df = pd.DataFrame()

In [20]:
df.index = bt['Date']

In [21]:
df['count'] = 0
df['score_mean'] = 0
df['score_max'] = 0
df['score_min'] = 0
df['score_pos_std'] = 0
df['score_neg_std'] = 0

In [22]:
for i in bt['Date']:
    array = news.loc[news['Date'] == i]['VaderScore'].to_numpy(dtype='float32')
    df['count'][i] = len(array)
    if len(array) != 0:
        df['score_mean'][i] = array.mean()
        df['score_max'][i] = array.max()
        df['score_min'][i] = array.min()
        df['score_pos_std'][i] = array[array > 0].std()
        df['score_neg_std'][i] = array[array < 0].std()

In [23]:
df = df.fillna(0)

In [24]:
df.to_csv('data/news_norm.csv',index=True)